In [70]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models

In [71]:
input_path = '/home/werewolf97/imtech/pro/data/'

In [72]:
input_path = Path(input_path)

In [73]:
text_data = []

In [74]:
input_path

PosixPath('/home/werewolf97/imtech/pro/data')

In [75]:
converted_path = Path.joinpath(input_path.parent, "converted_files")
if not os.path.exists(converted_path):
    os.mkdir(converted_path)
#print(type(self.input_path.as_posix()))
files = input_path.rglob(f"*.pdf")
#print("Files found: {0}".format(files))
exts = [".pdf", ".txt"]

In [76]:
for file in input_path.rglob('*'):
#     path = dirc+file
    with open(file, "r", encoding="utf-8") as f:
        content = f.read().replace('\n', ' ')
    if content is not None:
        data = [f'{file.parent.name}_{file.name}', content]
        text_data.append(data)
text_corpus = pd.DataFrame(text_data, columns=['resource_id', 'description'])

In [77]:
corpus = text_corpus

In [78]:
corpus['full_text'] = ""
corpus['full_text'] = corpus['description']

In [80]:
corpus

,resource_id,description,full_text
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar..."
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...
...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...


In [81]:
def strip_html_tags(text):
#     """Strips the HTML Tags from the text"""
    try:
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
    except:
        stripped_text = text
    return stripped_text

In [82]:
def remove_accented_chars(text):
#         """Remove Non UTF-8 Characters from text"""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [83]:
def expand_contractions(text, contraction_mapping=contractions.CONTRACTION_MAP):
#         """Replace contractions in string of text"""
        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                          flags=re.IGNORECASE | re.DOTALL)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match) \
                if contraction_mapping.get(match) \
                else contraction_mapping.get(match.lower())
            expanded_contraction = first_char + expanded_contraction[1:]
            return expanded_contraction

        try:
            expanded_text = contractions_pattern.sub(expand_match, text)
            expanded_text = re.sub("'", "", expanded_text)
        except:
            expanded_text = text
        return expanded_text

In [84]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [85]:
def lemmatize_text(text):
#         """Lemmatize the text using the WordNetLemmatizer"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = tokenizer.tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

In [86]:
def remove_special_characters(text, remove_digits):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [87]:
#to include google stoplist comment out the third last lin

def remove_stopwords( text):
#         """Remove Stopwords defined in the stopwords corpus"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    stopword_list = nltk.corpus.stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')
    stopword_list.append('cid')
    for c in ascii_uppercase:
        stopword_list.append(c)
    for c in ascii_lowercase:
        stopword_list.append(c)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    tokens = filtered_tokens
    stopword_list = google_list.stopwords
#     filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [88]:
def normalize_corpus(corpus):
    normalized_corpus = []
    counter = 0
    for i in range(len(corpus)):
        doc = corpus['description'][i]
        counter += 1
        doc = strip_html_tags(doc)
        doc = remove_accented_chars(doc)
        doc = expand_contractions(doc)
        doc = doc.lower()
        doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)
        doc = lemmatize_text(doc)
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_characters(doc, remove_digits= True)
        doc = re.sub(' +', ' ', doc)
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
    return normalized_corpus

In [89]:
corpus['clean_text'] = normalize_corpus(corpus)

In [90]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [91]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [92]:
corpus

,resource_id,description,full_text,clean_text,tokens
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,connectivity frank kammer hanjo aubig chapter ...,"[connectivity, frank, kammer, hanjo, aubig, ch..."
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,network science web mandate structural analysi...,"[network, science, web, mandate, structural, a..."
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality index dirk kosch utzki katharina an...,"[centrality, index, dirk, kosch, utzki, kathar..."
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,social learning network module network science...,"[social, learning, network, module, network, s..."
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,chapter matching market seen number way thinki...,"[chapter, matching, market, seen, number, way,..."
...,...,...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,chapter link analysis web search searching web...,"[chapter, link, analysis, web, search, searchi..."
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,network comparison michael baur marc benkert f...,"[network, comparison, michael, baur, marc, ben..."
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,introduction ulrik brandes thomas erlebach man...,"[introduction, ulrik, brandes, thomas, erlebac..."
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley value recap cooperating agent agent wi...,"[shapley, value, recap, cooperating, agent, ag..."
